# Step-by-step BCDI analysis 

## Table of contents
1. [Introduction](#introduction)
2. [Preprocessing](#preprocessing)
    1. [Loading the data](#loading)
    2. [Cropping the data](#cropping)
    3. [Cleaning the data](#cleaning)
3. [Phasing](#phasing)
    1. [Initialisation](#initialisation)
    2. [Running the phase retrieval](#running)
    3. [Phasing result analysis](#phasing_result_analysis)
4. [Orthogonalisation of the reconstructed data](#orthogonalisation)
    1. [Define the geometry associated to the beamline](#geometry)
    2. [Convention conversion](#convention)
5. [Extracting quantitative structural properties](#properties)
6. [Plotting](#plotting)
7. [Saving](#saving)

## **Introduction** <a name="introduction"></a>

This notebook provides a step-by-step guide for BCDI data analysis. For advanced users familiar with ***cdiutils*** parameters, a streamlined notebook is available. For high-throughput processing, use the provided scripts.

Commented code blocks throughout represent optional sanity checks—uncomment to visualise intermediate steps.

**Notes:**
* Some utility functions simplify plotting when dealing with multiple subplots. To inspect:
    * Function parameters: `function_name?`
    * Source code: `function_name??`

Import libraries:

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

import cdiutils
from cdiutils.interactive import Plotter

# update the matplotlib parameters
cdiutils.update_plot_params()

## **Preprocessing** <a name="preprocessing"></a>
### **Loading the data** <a name="loading"></a>

Inspect the data directory structure:

In [ ]:
experiment_path = ""  # the path to the /RAW_DATA dir of the experiment
# Ex: path = "/data/visitor/<experiment_name>/id01/<date>/RAW_DATA/"

!tree -L 1 {experiment_path}

In [ ]:
experiment_name = ""  # required
experiment_file_path = f"{experiment_path}/{experiment_name}_id01.h5"
print(f"Experiment file path: {experiment_file_path}")

bliss_sample_name = ""  # required
bliss_dataset_name = ""  # required
scan = None  # required

sample_name = f"{bliss_sample_name}_{bliss_dataset_name}"


loader = cdiutils.io.ID01Loader(
    experiment_file_path=experiment_file_path,
    sample_name=sample_name,
)
data = loader.load_detector_data(scan)

print(f"Shape of the detector data is: {data.shape}")
plt.figure(layout="tight")
plt.imshow(data[data.shape[0] // 2], norm="log")
plt.colorbar()
plt.show()

Interactive slider visualisation of the detector data:

In [ ]:
Plotter(data, plot="2D");

### **Cropping the data** <a name="cropping"></a>

Locate the Bragg peak position:

In [ ]:
# data.shape[0] represents number of frames in rocking curve
output_shape = (data.shape[0], 150, 150)

# crop data using chain_centring with specified methods
# methods: list of "com"/"max" strings or tuple for reference pixel
# note: hot pixels may cause "max" or "com" to fail
(
    cropped_data,  # output cropped data
    det_ref,  # detector reference voxel in full detector frame
    cropped_det_ref,  # detector reference voxel in cropped frame
    roi,  # region of interest (ROI) used to crop data
) = cdiutils.utils.CroppingHandler.chain_centring(
    data,
    methods=["max", "com"],  # methods used sequentially
    output_shape=output_shape,  # target output shape
    verbose=True,  # print logs during reference voxel search
)

# plot cropped detector data
loader.plot_detector_data(cropped_data, f"Scan #{scan}", equal_limits=False)

### **Cleaning the data** <a name="cleaning"></a>

Clean the data to improve analysis quality:

* Remove hot pixels using `cdiutils.utils.hot_pixel_filter` (median filter-based).
* Manual removal: enable interactive mode with `%matplotlib ipympl`, then revert with `%matplotlib inline`.
* Apply flat-field correction if available.
* Subtract background for fluorescence suppression.

In [ ]:
# apply flat field if available (at correct energy)
# flat_field_path = ""

# with np.load(flat_field_path) as file:
#     flat_field = file["arr_0"][
#         cdiutils.utils.CroppingHandler.roi_list_to_slices(roi[2:])
#     ]
# cleaned_data = cropped_data * flat_field

# remove hot pixels using median filter-based method
# cleaned_data = cropped_data
cleaned_data, hot_pixel_mask = cdiutils.utils.hot_pixel_filter(cropped_data)

# subtract background (e.g., fluorescence)
background_level = 4
cleaned_data = np.where(
    cleaned_data - background_level > 0, cleaned_data - background_level, 0
)

# plot cleaned detector data
loader.plot_detector_data(cleaned_data, f"Scan #{scan}")

Load the detector mask with correct ROI:

In [ ]:
mask = cdiutils.io.Loader.get_mask(
    detector_name="",  # required,
    channel=cleaned_data.shape[
        0
    ],  # or data.shape[0] depending on the cropping
    roi=roi,
)
# mask *= hot_pixel_mask
cdiutils.plot.plot_volume_slices(mask, title=f"Mask, scan #{scan}");

## **Phasing** <a name="phasing"></a>

> Requires PyNX

### **Initialisation** <a name="initialisation"></a>

In [ ]:
# Optional: Load support from previous analysis
# good_run_path = f"results/{sample_name}/S{scan}/pynx_phasing/.cxi"
# with cdiutils.io.CXIFile(run_path) as file:
#     good_support = file["entry_1/image_1/support"]

In [ ]:
# initialise PyNXPhaser wrapper for quick PyNX setup
# required: iobs (observed intensity) and mask
# most parameters have defaults; to inspect:
# - print(phaser)
# - print(cdiutils.process.phaser.DEFAULT_PYNX_PARAMS)
# - PyNX API: https://pynx.esrf.fr/en/latest/modules/cdi/index.html

params = {
    "support_update_period": 50,
    "support_threshold": "0.15, 0.40",
    "support_autocorrelation_threshold": (0.05, 0.11),
    "show_cdi": 0,
    # "update_border_n": 4,
    # "post_expand": (-1, 1),
    "rebin": "1, 1, 1",
    "scale_obj": "I",
    # "support_shape": "square",
    # "support_size": 20,
}

phaser = cdiutils.process.PyNXPhaser(iobs=cleaned_data, mask=mask, **params)

# initialise CDI object (optional: provide support or previous cdi)
phaser.init_cdi(
    # support=good_support,
)

# plot initial guess
phaser.plot_cdi(phaser.cdi);

### **Running the phase retrieval** <a name="running"></a>

In [ ]:
# define phase retrieval recipe
recipe = "HIO**400, RAAR**500, ER**200, FAA"
# FAA stands for Fourier Apply Amplitude, meaning, a forward FFT,
# amplitude constraint, inverse FFT without any real space constraint

phaser.run_multiple_instances(run_nb=5, recipe=recipe)

# genetic phasing requires smaller recipes
# recipe = "HIO**50, RAAR**60, ER**40"
# phaser.genetic_phasing(
#     run_nb=5,
#     genetic_pass_nb=10,
#     recipe=recipe,
#     selection_method="mean_to_max"
# )

Preview final results (detailed analysis follows):

In [ ]:
for i, cdi in enumerate(phaser.cdi_list):
    phaser.plot_cdi(cdi, title=f"Run {i + 1:04d}")

Select a specific run to restart phasing (optional):

In [ ]:
# good_run = 3
# good_support = phaser.cdi_list[good_run - 1].get_support(shift=True)

### **Phasing result analysis** <a name="phasing_result_analysis"></a>

The `PhasingResultAnalyser` class provides methods for sorting and analyzing phase retrieval results.

`analyse_phasing_results` sorts by `sorting_criterion`:
* `mean_to_max`: Difference between Gaussian mean and max amplitude (lower = more homogeneous)
* `sharpness`: Sum of amplitude^4 within support (lower = more homogeneous for similar supports)
* `std`: Standard deviation of amplitude
* `llk`: Log-likelihood
* `llkf`: Free log-likelihood

In [ ]:
analyser = cdiutils.process.PhasingResultAnalyser(cdi_results=phaser.cdi_list)

analyser.analyse_phasing_results(
    sorting_criterion="mean_to_max"
    # plot_phasing_results=False,  # Defaults to True
    # plot_phase=True,  # Defaults to False
)

#### Select best candidates and run mode decomposition

Two selection options:
- **Manual**: Define `best_runs` as a list of run numbers based on visual inspection.
- **Automatic**: Set `nb_of_best_sorted_runs` to auto-select using the sorting criterion.

Mode decomposition (similar to PCA) extracts the principal mode from selected runs.

In [ ]:
analyser.select_best_candidates(
    # best_runs=[2, 5]
    nb_of_best_sorted_runs=3,
)
print(f"The best candidates selected are: {analyser.best_candidates}.")
modes, mode_weight = analyser.mode_decomposition()

mode = modes[0]  # Select the first mode

#### Check the amplitude distribution

`cdiutils.analysis.find_isosurface` estimates an isosurface from the amplitude histogram.

Assuming a Gaussian right tail, the isosurface is defined as $\mu - 3\sigma$ (mean minus 3× standard deviation).

**Note:** Computation may be unstable—use the plot as a guide.

In [ ]:
isosurface, _ = cdiutils.analysis.find_isosurface(np.abs(mode), plot=True)

#### Define support and calculate oversampling ratio

1. Define `support` array matching object morphology.
2. Calculate oversampling ratio per direction.
3. Use ratios to optimize PyNX `rebin` parameter for re-phasing.

In [ ]:
# isosurface = 0.30
support = cdiutils.utils.make_support(np.abs(mode), isosurface=isosurface)

# calculate oversampling ratios per direction
ratios = cdiutils.utils.get_oversampling_ratios(support)
print(
    "[INFO] The oversampling ratios in each direction are "
    + ", ".join([f"axis{i}: {ratios[i]:.1f}" for i in range(len(ratios))])
    + ".\nIf low-strain crystal, you can set PyNX 'rebin' parameter to "
    "(" + ", ".join([f"{r // 2}" for r in ratios]) + ")"
)

Visualise amplitude and phase of the final reconstruction:

In [ ]:
figure, axes = plt.subplots(2, 3, layout="tight", figsize=(6, 4))

# get centred slices for 3D visualisation
slices = cdiutils.utils.get_centred_slices(mode.shape)
for i in range(3):
    amp_img = axes[0, i].imshow(np.abs(mode)[slices[i]])
    phase_img = axes[1, i].imshow(
        np.angle(mode)[slices[i]],
        cmap="cet_CET_C9s_r",
        alpha=(np.abs(mode) / np.max(np.abs(mode)))[slices[i]],
    )

    for ax in (axes[0, i], axes[1, i]):
        cdiutils.plot.add_colorbar(ax, ax.images[0])
        limits = cdiutils.plot.x_y_lim_from_support(support[slices[i]])
        ax.set_xlim(limits[0])
        ax.set_ylim(limits[1])

## **Orthogonalisation of the reconstructed data** <a name="orthogonalisation"></a>

Transform from detector frame to XU/CXI lab frame:
- Retrieve motor positions from data file.
- Build reciprocal space grid.
- Calculate transformation matrices.

In [ ]:
angles = loader.load_motor_positions(scan, roi=roi)
energy = loader.load_energy(scan)  # or define it manually.

### Initialise the reciprocal space grid

Detector calibration parameters (`det_calib_params`) required:
- Direct beam position: `cch1` (vertical), `cch2` (horizontal)
- Pixel size: `pwidth1`, `pwidth2`
- Sample-to-detector distance: `distance`

Additional parameters improve accuracy.

In [ ]:
det_calib_params = loader.load_det_calib_params(scan)  # beamline-dependent

# manual detector calibration parameters:
# det_calib_params = {
#     "cch1": ,  # direct beam vertical position
#     "cch2": ,  # horizontal position
#     "pwidth1": 5.5e-05,  # pixel size (m), Eiger: 7.5e-5, Maxipix: 5.5e-5
#     "pwidth2": 5.5e-05,
#     "distance": ,  # sample-detector distance (m)
#     "tiltazimuth": 0.0,
#     "tilt": 0.0,
#     "detrot": 0.0,
#     "outerangle_offset": 0.0,
# }

### **Define the geometry** <a name="geometry"></a>

Inspect: `print(geometry)`

In [ ]:
# load appropriate geometry
geometry = cdiutils.Geometry.from_setup("ID01")

# initialise space converter
converter = cdiutils.SpaceConverter(
    geometry, det_calib_params, energy=energy, roi=roi[2:]
)

# Q space area initialised only for selected ROI used before cropping
converter.init_q_space(**angles)

#### Verify Q-space gridding

Requires Bragg reflection and lattice parameter information.

In [ ]:
# Specify measured Bragg reflection
hkl = [1, 1, 1]

In [ ]:
# cropped_det_ref is pixel reference chosen at beginning
# it is the centre of cropped data
q_lab_ref = converter.index_det_to_q_lab(cropped_det_ref)
dspacing_ref = converter.dspacing(q_lab_ref)
lattice_parameter_ref = converter.lattice_parameter(q_lab_ref, hkl)
print(
    f"The d-spacing and 'effective' lattice parameter are respectively "
    f"{dspacing_ref:.4f} and {lattice_parameter_ref:.4f} angstroms.\n"
    "Is that what you expect?! -> If not, the detector calibration might "
    "be wrong."
)

Initialize interpolators for reciprocal and direct space orthogonalization:

In [ ]:
converter.init_interpolator(space="both", verbose=True)

In [ ]:
# This is the orthogonalised intensity
ortho_intensity = converter.orthogonalise_to_q_lab(cleaned_data)

# This is the regular Q-space grid
qx, qy, qz = converter.get_q_lab_regular_grid()

Visualise intensity in orthogonal Q-space:

In [ ]:
q_spacing = [np.mean(np.diff(q)) for q in (qx, qy, qz)]
q_centre = (qx.mean(), qy.mean(), qz.mean())

figure, axes = cdiutils.plot.slice.plot_volume_slices(
    ortho_intensity,
    voxel_size=q_spacing,
    data_centre=q_centre,
    title="Orthogonalised intensity in the Q-lab frame",
    norm="log",
    convention="xu",
    show=False,
)
cdiutils.plot.add_labels(axes, space="rcp", convention="xu")
display(figure)

#### Orthogonalisation in direct space

Specify voxel size (float, tuple, list, or np.ndarray in nm). If unspecified, uses previously determined size.

In [ ]:
# use automatically determined voxel size or define manually
# manual definition triggers interpolation
voxel_size = converter.direct_lab_voxel_size  # auto
# voxel_size = 15  # manual (nm)

ortho_obj = converter.orthogonalise_to_direct_lab(mode, voxel_size)
voxel_size = converter.direct_lab_voxel_size
print(f"The actual voxel size is: {voxel_size} nm.")

In [ ]:
isosurface, _ = cdiutils.analysis.find_isosurface(np.abs(ortho_obj), plot=True)

In [ ]:
# isosurface = 0.30  # Define manually if estimated value is unsatisfactory
ortho_support = cdiutils.utils.make_support(np.abs(ortho_obj), isosurface)

In [ ]:
figures = {}
axes = {}

figures["amp"], axes["amp"] = cdiutils.plot.plot_volume_slices(
    np.abs(ortho_obj),
    support=ortho_support,
    voxel_size=voxel_size,
    data_centre=(0, 0, 0),
    convention="xu",
    title="Amplitude",
    show=False,
)
cdiutils.plot.add_labels(axes["amp"], space="direct", convention="xu")

figures["phase"], axes["phase"] = cdiutils.plot.plot_volume_slices(
    np.angle(ortho_obj) * ortho_support,
    support=ortho_support,
    data_centre=(0, 0, 0),
    voxel_size=voxel_size,
    cmap="cet_CET_C9s_r",
    convention="xu",
    vmin=-np.pi,
    vmax=np.pi,
    title="Phase (rad)",
    show=False,
)
cdiutils.plot.add_labels(axes["phase"], space="direct", convention="xu")

display(figures["amp"], figures["phase"])

### **Convention conversion** <a name="convention"></a>

The CXI convention (https://cxidb.org/cxi.html) is widely adopted. `cdiutils` provides conversion from *xrayutilities* (XU) to CXI.

**XU convention:**
* Data order: $[\text{axis}_{0} = x_{\text{XU}}, \text{axis}_{1} = y_{\text{XU}}, \text{axis}_{2} = z_{\text{XU}}]$
* $x_{\text{XU}}$: beam direction (away from source)
* $y_{\text{XU}}$: outboard (horizontal)
* $z_{\text{XU}}$: vertical up

**CXI convention:**
* Data order: $[\text{axis}_{0} = z_{\text{CXI}}, \text{axis}_{1} = y_{\text{CXI}}, \text{axis}_{2} = x_{\text{CXI}}]$
* $x_{\text{CXI}}$: horizontal (right-handed)
* $y_{\text{CXI}}$: vertical up
* $z_{\text{CXI}}$: beam direction

![XU_and_CXI](https://github.com/clatlan/cdiutils/assets/38456566/5db91309-0735-4910-9090-5299666f6994)

In [ ]:
cxi_ortho_obj = geometry.swap_convention(ortho_obj)
cxi_ortho_support = geometry.swap_convention(ortho_support)
cxi_voxel_size = geometry.swap_convention(voxel_size)

## **Extracting quantitative structural properties** <a name="properties"></a>

The `PostProcessor` class provides static methods for post-phasing analysis:

- `flip_reconstruction`: Correct complex conjugate solutions
- `apodize`: Suppress high-frequency artifacts via 3D windowing (blackman, hamming, hann, gaussian, etc.)
- `unwrap_phase`: Phase unwrapping within support (`skimage.restoration.unwrap_phase`)
- `remove_phase_ramp`: Linear regression-based 3D phase ramp removal
- `phase_offset_to_zero`: Center phase distribution at zero
- `get_displacement`: Displacement calculation from phase and Bragg peak position
- `get_het_normal_strain`: Heterogeneous normal strain extraction:
    * Traditional `numpy.gradient` (loses surface voxels)
    * Hybrid gradient (2nd-order bulk, 1st-order surface—preserves surface accuracy)

**Unified method:** `get_structural_properties` generates:
* Displacement maps
* Strain maps
* d-spacing maps
* Lattice parameter maps

Optionally flip and/or apodize the reconstruction:

In [ ]:
# cxi_ortho_obj = cdiutils.process.PostProcessor.flip_reconstruction(cxi_ortho_obj)
cxi_ortho_obj = cdiutils.process.PostProcessor.apodize(
    cxi_ortho_obj, "blackman"
)

Extract structural properties:

In [ ]:
struct_props = cdiutils.process.PostProcessor.get_structural_properties(
    cxi_ortho_obj,
    isosurface=isosurface,
    g_vector=geometry.swap_convention(q_lab_ref),
    hkl=hkl,
    voxel_size=cxi_voxel_size,
    handle_defects=False,  # this is whether you expect a defect.
)

for prop, value in struct_props.items():
    print(f"{prop}: ", end="")
    if isinstance(value, (np.ndarray)) and value.ndim > 1:
        print(f"3D array of shape: {value.shape}")
    elif isinstance(value, (list, tuple)):
        if isinstance(value[0], np.ndarray):
            print(f"tuple or list of length = {len(value)}")
        else:
            print(value)
    else:
        print(value)

## **Plotting** <a name="plotting"></a>

### Summary plot

In [ ]:
to_plot = {
    k: struct_props[k]
    for k in [
        "amplitude",
        "phase",
        "displacement",
        "het_strain",
        "lattice_parameter",
    ]
}

table_info = {
    "Isosurface": isosurface,
    "Averaged Lat. Par. (Å)": np.nanmean(struct_props["lattice_parameter"]),
    "Averaged d-spacing (Å)": np.nanmean(struct_props["dspacing"]),
}

summary_fig = cdiutils.pipeline.PipelinePlotter.summary_plot(
    title=f"Summary figure, Scan #{scan}",
    support=struct_props["support"],
    table_info=table_info,
    voxel_size=cxi_voxel_size,
    **to_plot,
)

### 3D surface strain projections

In [ ]:
fig = cdiutils.plot.volume.plot_3d_surface_projections(
    data=struct_props["het_strain"],
    support=struct_props["support"],
    voxel_size=cxi_voxel_size,
    cmap="cet_CET_D13",
    vmin=-np.nanmax(np.abs(struct_props["het_strain"])),
    vmax=np.nanmax(np.abs(struct_props["het_strain"])),
    cbar_title=r"Strain (%)",
    title=f"3D views of the strain, Scan #{scan}",
)

### Interactive 3D isosurface visualisation

In [ ]:
from cdiutils.interactive import plot_3d_isosurface

In [ ]:
quantities_to_be_visualised = {
    k: struct_props[k]
    for k in [
        "amplitude",
        "support",
        "phase",
        "displacement",
        "het_strain",
        "het_strain_from_dspacing",
        "lattice_parameter",
        "dspacing",
    ]
}

plot_3d_isosurface(
    quantities_to_be_visualised["amplitude"],
    quantities_to_be_visualised,
    voxel_size=cxi_voxel_size,
    initial_quantity="het_strain_from_dspacing",
    cmap="cet_CET_D13",
)

## **Saving** <a name="saving"></a>

In [ ]:
# Specify output directory
dump_dir = f"results/{sample_name}/S{scan}_step_by_step/"

if os.path.isdir(dump_dir):
    print("[INFO] Dump directory exists:", dump_dir)
else:
    print(f"[INFO] Creating dump directory: {dump_dir}")
    os.makedirs(dump_dir, exist_ok=True)

# Prepare data for saving
to_save = {
    "isosurface": isosurface,
    "q_lab_reference": q_lab_ref,
    "dspacing_reference": dspacing_ref,
    "lattice_parameter_reference": lattice_parameter_ref,
}
to_save.update(struct_props)

# Save as .npz
np.savez(f"{dump_dir}/S{scan}_structural_properties.npz", **to_save)

# Prepare VTI export (for 3D visualisation software)
# Replace NaN with mean (avoids visualisation artifacts)
for k in (
    "het_strain",
    "het_strain_from_dspacing",
    "dspacing",
    "lattice_parameter",
    "displacement",
):
    quantities_to_be_visualised[k] = np.where(
        np.isnan(quantities_to_be_visualised[k]),
        np.nanmean(quantities_to_be_visualised[k]),
        quantities_to_be_visualised[k],
    )

cdiutils.io.save_as_vti(
    f"{dump_dir}/S{scan}_structural_properties.vti",
    voxel_size=cxi_voxel_size,
    cxi_convention=True,
    **quantities_to_be_visualised,
)

print("\n[INFO] Data saved.")

### Alternative visualisation

You can also visualise the data using the `plot_volume_slices` function (see cell below).

For further post-processing analysis and comparing multiple BCDI results, check out this [notebook example](https://github.com/clatlan/cdiutils/blob/master/examples/bcdi_reconstruction_analysis.ipynb).

In [ ]:
_, _, plot_configs = cdiutils.plot.set_plot_configs()
for prop in (
    "amplitude",
    "support",
    "phase",
    "displacement",
    "het_strain",
    "dspacing",
):
    figures[prop], axes[prop] = cdiutils.plot.slice.plot_volume_slices(
        struct_props[prop]
        * cdiutils.utils.zero_to_nan(struct_props["support"]),
        support=struct_props["support"],
        voxel_size=cxi_voxel_size,
        data_centre=(0, 0, 0),
        vmin=plot_configs[prop]["vmin"],
        vmax=plot_configs[prop]["vmax"],
        cmap=plot_configs[prop]["cmap"],
        title=prop,
        show=False,
    )
    cdiutils.plot.add_labels(axes[prop])
    display(figures[prop])

---

**Feedback & Issues:**
- Email: [clement.atlan@esrf.fr](mailto:clement.atlan@esrf.fr?subject=cdiutils)
- GitHub: [cdiutils/issues](https://github.com/clatlan/cdiutils/issues)

## Credits

This notebook is part of the `cdiutils` package (Clément Atlan, ESRF, 2025).

**Citation:**
```bibtex
@software{Atlan_Cdiutils_A_python,
  author = {Atlan, Clement},
  doi = {10.5281/zenodo.7656853},
  license = {MIT},
  title = {{Cdiutils: A python package for Bragg Coherent Diffraction 
           Imaging processing, analysis and visualisation workflows}},
  url = {https://github.com/clatlan/cdiutils},
  version = {0.2.0}
}
```